<a href="https://colab.research.google.com/github/xin-kai08/Machine-Learning-Models/blob/main/machine_learning_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# 掛載 Google 雲端硬碟
drive.mount('/content/drive')

# 資料集根目錄
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/dataset/feature dim_4"

# 各分類資料夾設定
LABEL_DIRS = {
    0: os.path.join(BASE_PATH, "normal"),
    1: os.path.join(BASE_PATH, "abnormal/transformer_rust"),
    2: os.path.join(BASE_PATH, "abnormal/wire_rust"),
    3: os.path.join(BASE_PATH, "abnormal/wire_peeling"),
    4: os.path.join(BASE_PATH, "abnormal/wire_bending"),
    5: os.path.join(BASE_PATH, "abnormal/phone_overheating"),
}

Mounted at /content/drive


定義快取函數(三維)

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def generate_preprocessed_cache_3d(seq_lens, label_dirs, cache_dir="/content/preprocessed"):
    os.makedirs(cache_dir, exist_ok=True)

    for seq_len in seq_lens:
        cache_X = os.path.join(cache_dir, f"X_seq{seq_len}_3d.npy")
        cache_y = os.path.join(cache_dir, f"y_seq{seq_len}_3d.npy")

        if os.path.exists(cache_X) and os.path.exists(cache_y):
            print(f"📥 已存在 3D 快取：seq_len={seq_len}，略過")
            continue

        all_seq, all_labels = [], []
        for label, folder in label_dirs.items():
            for fname in os.listdir(folder):
                if fname.endswith(".csv"):
                    path = os.path.join(folder, fname)
                    df = pd.read_csv(path)
                    data = df[["voltage", "current", "power", "temp_C"]].values.astype(np.float32)
                    num_chunks = len(data) // seq_len
                    chunks = [data[i * seq_len : (i + 1) * seq_len] for i in range(num_chunks)]
                    all_seq.extend(chunks)
                    all_labels.extend([label] * len(chunks))

        seq_arr = np.array(all_seq, dtype=np.float32)
        labels_arr = np.array(all_labels, dtype=np.int64)
        B, T, F = seq_arr.shape
        reshaped = seq_arr.reshape(-1, F)
        scaled = StandardScaler().fit_transform(reshaped).reshape(B, T, F)

        np.save(cache_X, scaled)
        np.save(cache_y, labels_arr)
        print(f"✅ 完成 3D 快取：seq_len={seq_len}（共 {B} 筆序列）")

定義快取函數(二維)

In [ ]:
def generate_preprocessed_cache_2d(seq_lens, label_dirs, cache_dir="/content/preprocessed"):
    os.makedirs(cache_dir, exist_ok=True)

    for seq_len in seq_lens:
        cache_X = os.path.join(cache_dir, f"X_seq{seq_len}_2d.npy")
        cache_y = os.path.join(cache_dir, f"y_seq{seq_len}_2d.npy")

        if os.path.exists(cache_X) and os.path.exists(cache_y):
            print(f"📥 已存在 2D 快取：seq_len={seq_len}，略過")
            continue

        all_features, all_labels = [], []
        for label, folder in label_dirs.items():
            for fname in os.listdir(folder):
                if fname.endswith(".csv"):
                    path = os.path.join(folder, fname)
                    df = pd.read_csv(path)
                    data = df[["voltage", "current", "power", "temp_C"]].values.astype(np.float32)
                    num_chunks = len(data) // seq_len
                    chunks = [data[i * seq_len : (i + 1) * seq_len] for i in range(num_chunks)]

                    for chunk in chunks:
                        features = []
                        features.extend(np.mean(chunk, axis=0))  # 3
                        features.extend(np.std(chunk, axis=0))   # 3
                        features.extend(np.max(chunk, axis=0))   # 3
                        features.extend(np.min(chunk, axis=0))   # 3
                        all_features.append(features)
                        all_labels.append(label)

        X = np.array(all_features, dtype=np.float32)
        y = np.array(all_labels, dtype=np.int64)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        np.save(cache_X, X_scaled)
        np.save(cache_y, y)
        print(f"✅ 完成 2D 快取：seq_len={seq_len}（共 {len(X_scaled)} 筆）")

實際執行快取

In [ ]:
seq_lens = [4, 8, 10, 20, 30, 40]
generate_preprocessed_cache_3d(seq_lens, LABEL_DIRS)      # 產生 3D
generate_preprocessed_cache_2d(seq_lens, LABEL_DIRS)   # 產生 2D

✅ 完成 3D 快取：seq_len=4（共 15458 筆序列）
✅ 完成 3D 快取：seq_len=8（共 7723 筆序列）
✅ 完成 3D 快取：seq_len=10（共 6172 筆序列）
✅ 完成 3D 快取：seq_len=20（共 3077 筆序列）
✅ 完成 3D 快取：seq_len=30（共 2047 筆序列）
✅ 完成 3D 快取：seq_len=40（共 1530 筆序列）
✅ 完成 2D 快取：seq_len=4（共 15458 筆）
✅ 完成 2D 快取：seq_len=8（共 7723 筆）
✅ 完成 2D 快取：seq_len=10（共 6172 筆）
✅ 完成 2D 快取：seq_len=20（共 3077 筆）
✅ 完成 2D 快取：seq_len=30（共 2047 筆）
✅ 完成 2D 快取：seq_len=40（共 1530 筆）


LSTM

In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/LSTM"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === LSTM 模型 ===
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

# === 主訓練與超參數搜尋函數（使用快取） ===
def train_and_search_lstm(batch_sizes, learning_rates, seq_lens,
                          num_epochs=100, hidden_dim=64, num_layers=1,
                          num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "/content/preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = LSTMClassifier(input_dim=3, hidden_dim=hidden_dim,
                                           num_layers=num_layers, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric_per_fold(fold_lists, metric_name, folder, num_epochs, bs, lr, seq_len):
                    plt.figure(figsize=(10, 6))
                    for fold_idx, fold_metric in enumerate(fold_lists):
                        plt.plot(range(1, num_epochs + 1), fold_metric, label=f"Fold {fold_idx + 1}")  # 純線條，沒有 marker

                    plt.title(f"Combined {metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    if metric_name != 'loss':
                        plt.ylim(0, 1.0)
                    plt.grid(True)
                    plt.legend()
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}_combined_{metric_name}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                # === Plot ===
                plot_metric_per_fold(acc_curves, "accuracy", "accuracy_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(loss_curves, "loss", "loss_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(f1_curves, "f1_score", "f1_score_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(acc_train_curves, "accuracy", "train_accuracy", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(loss_train_curves, "loss", "train_loss", num_epochs, bs, lr, seq_len)

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "lstm_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [ ]:
# 設定訓練參數
batch_sizes = [4, 8, 16, 32]
learning_rates = [1e-2, 1e-3, 1e-4]
seq_lens = [4, 8, 10, 20, 30, 40]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_lstm(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")#
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

🔍 總共訓練組數：72，每組訓練 100 epochs

🧪 BS=4 | LR=0.01 | SEQ=4
    [Fold 1] Epoch 10/100 | Acc: 0.9812 | Loss: 0.0769
    [Fold 1] Epoch 20/100 | Acc: 0.9867 | Loss: 0.0628
    [Fold 1] Epoch 30/100 | Acc: 0.9891 | Loss: 0.0412
    [Fold 1] Epoch 40/100 | Acc: 0.9901 | Loss: 0.0517
    [Fold 1] Epoch 50/100 | Acc: 0.9881 | Loss: 0.0517
    [Fold 1] Epoch 60/100 | Acc: 0.9876 | Loss: 0.0578
    [Fold 1] Epoch 70/100 | Acc: 0.9852 | Loss: 0.0686
    [Fold 1] Epoch 80/100 | Acc: 0.9906 | Loss: 0.0323
    [Fold 1] Epoch 90/100 | Acc: 0.9896 | Loss: 0.0486
    [Fold 1] Epoch 100/100 | Acc: 0.9876 | Loss: 0.0590
    [Fold 2] Epoch 10/100 | Acc: 0.9916 | Loss: 0.0390
    [Fold 2] Epoch 20/100 | Acc: 0.9921 | Loss: 0.0393
    [Fold 2] Epoch 30/100 | Acc: 0.9886 | Loss: 0.0496
    [Fold 2] Epoch 40/100 | Acc: 0.9886 | Loss: 0.0497
    [Fold 2] Epoch 50/100 | Acc: 0.9941 | Loss: 0.0321
    [Fold 2] Epoch 60/100 | Acc: 0.9822 | Loss: 0.0693
    [Fold 2] Epoch 70/100 | Acc: 0.9916 | Loss: 0.0421
    [Fold 

FileNotFoundError: [Errno 2] No such file or directory: '/content/preprocessed/X_seq20_3d.npy'

MLP

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/MLP"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === MLP 模型 ===
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.model(x)

# === 訓練與儲存 ===
def train_and_search_mlp(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, hidden_dim=128,
                         num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_2d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_2d.npy")
                X = np.load(x_path)
                y = np.load(y_path)
                input_dim = X.shape[1]

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = MLPClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric_per_fold(fold_lists, metric_name, folder, num_epochs, bs, lr, seq_len):
                    plt.figure(figsize=(10, 6))
                    for fold_idx, fold_metric in enumerate(fold_lists):
                        plt.plot(range(1, num_epochs + 1), fold_metric, label=f"Fold {fold_idx + 1}")  # 純線條

                    plt.title(f"Combined {metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    if metric_name != 'loss':
                        plt.ylim(0, 1.0)
                    plt.grid(True)
                    plt.legend()
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}_combined_{metric_name}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                # === Plot ===
                plot_metric_per_fold(acc_curves, "accuracy", "accuracy_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(loss_curves, "loss", "loss_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(f1_curves, "f1_score", "f1_score_curves", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(acc_train_curves, "accuracy", "train_accuracy", num_epochs, bs, lr, seq_len)
                plot_metric_per_fold(loss_train_curves, "loss", "train_loss", num_epochs, bs, lr, seq_len)


                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "mlp_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [ ]:
# 設定訓練參數
batch_sizes = [4, 8, 16, 32]
learning_rates = [1e-2, 1e-3, 1e-4]
seq_lens = [4, 8, 10, 20, 30, 40]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_mlp(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")


SVM

In [ ]:
# === SVM 訓練程式（改寫為與 LSTM 架構一致）===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/SVM"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 改寫後的 SVM 訓練流程 ===
def train_svm_search(kernels, Cs, seq_lens, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"
    results = []
    best = None

    for kernel in kernels:
        for C_val in Cs:
            for seq_len in seq_lens:
                print(f"\n🔎 Kernel={kernel} | C={C_val} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_2d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_2d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
                acc_list, f1_list, all_y_true, all_y_pred = [], [], [], []
                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]
                    clf = SVC(kernel=kernel, C=C_val)
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_val)
                    acc = accuracy_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)
                    acc_list.append(acc)
                    f1_list.append(f1)
                    all_y_true.extend(y_val)
                    all_y_pred.extend(y_pred)
                t1 = time.time()

                final_acc = np.mean(acc_list)
                final_f1 = np.mean(f1_list)
                if final_acc >= 1.0:
                    print(f"⚠️ Skipped Kernel={kernel} C={C_val} SEQ={seq_len} due to acc=1.0")
                    continue

                precision = precision_score(all_y_true, all_y_pred, average='macro', zero_division=0)
                recall = recall_score(all_y_true, all_y_pred, average='macro', zero_division=0)

                # 新增三種圖
                # 1. K-fold validation accuracy 曲線
                plt.figure()
                plt.plot(range(1, k_folds+1), acc_list, label='Validation Accuracy')
                plt.title(f"K-fold Accuracy\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.xlabel("Fold")
                plt.ylabel("Accuracy")
                plt.ylim(0, 1.01)
                plt.grid(True)
                plt.savefig(os.path.join(RESULT_DIR, "accuracy_curves", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()
                # 2. K-fold F1-score 曲線
                plt.figure()
                plt.plot(range(1, k_folds+1), f1_list, label='Validation F1-score', color='orange')
                plt.title(f"K-fold F1-score\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.xlabel("Fold")
                plt.ylabel("F1-score")
                plt.ylim(0, 1.01)
                plt.grid(True)
                plt.savefig(os.path.join(RESULT_DIR, "f1_score_curves", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()
                # 3. 訓練集準確率（整體）
                clf_full = SVC(kernel=kernel, C=C_val)
                clf_full.fit(X, y)
                train_pred = clf_full.predict(X)
                train_acc = accuracy_score(y, train_pred)
                plt.figure()
                plt.bar(['train'], [train_acc])
                plt.title(f"Train Accuracy\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.ylim(0, 1.01)
                plt.ylabel("Accuracy")
                plt.savefig(os.path.join(RESULT_DIR, "train_accuracy", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"k{kernel}_c{C_val}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                result = {
                    'kernel': kernel, 'C': C_val, 'seq_len': seq_len,
                    'final_acc': final_acc, 'precision': precision,
                    'recall': recall, 'f1_score': final_f1,
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(result)
                if best is None or result['final_acc'] > best['final_acc']:
                    best = result
                print(f"\n📊 統計結果 (Kernel={kernel}, C={C_val}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_acc:.4f}")
                print(f"  🔹 Precision      : {precision:.4f}")
                print(f"  🔹 Recall         : {recall:.4f}")
                print(f"  🔹 F1-score       : {final_f1:.4f}")
                print(f"  ⏱️  Training Time  : {round(t1 - t0, 2)} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "svm_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 Results saved to {csv_path}")
    return results, best

In [ ]:
seq_lens = [4, 5, 8, 10, 20, 30, 40]
C_list = [1.0, 10.0]
kernel_list = ["rbf", "linear"]

results, best = train_svm_search(kernel_list, C_list, seq_lens)

print("\n🎯 最佳參數組合：")
print(f"Sequence Length = {best['seq_len']}")
print(f"C = {best['C']}")
print(f"Kernel = {best['kernel']}")
print(f"Accuracy = {best['final_acc']:.4f}")
print(f"Precision = {best['precision']:.4f}")
print(f"Recall = {best['recall']:.4f}")
print(f"F1-score = {best['f1_score']:.4f}")
print(f"Training Time = {best['training_time_s']} 秒")

GRU

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/GRU"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === GRU 模型 ===
class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        return self.fc(out)

# === 訓練與儲存 ===
def train_and_search_gru(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, hidden_dim=64, num_layers=1, num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                X = np.load(os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy"))
                y = np.load(os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy"))

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = GRUClassifier(input_dim=3, hidden_dim=hidden_dim, num_layers=num_layers, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        model.train()
                        correct_train, total_train, total_loss_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train_list.append(correct_train / total_train)
                        loss_train_list.append(total_loss_train / total_train)

                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc_list.append(correct / total)
                        loss_list.append(total_loss / total)
                        f1_list.append(f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0))

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc_list[-1]:.4f} | Loss: {loss_list[-1]:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()
                def plot_metric_per_fold(fold_lists, metric_name, folder, bs, lr, seq_len, num_epochs):
                    plt.figure(figsize=(10, 6))
                    for fold_idx, fold_metric in enumerate(fold_lists):
                        plt.plot(range(1, num_epochs + 1), fold_metric, label=f"Fold {fold_idx + 1}")  # 純線條，無 marker

                    plt.title(f"Combined {metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    if metric_name != 'loss':
                        plt.ylim(0, 1.0)
                    plt.grid(True)
                    plt.legend()
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}_combined_{metric_name}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                plot_metric_per_fold(acc_curves, "accuracy", "accuracy_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_curves, "loss", "loss_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(f1_curves, "f1_score", "f1_score_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(acc_train_curves, "accuracy", "train_accuracy", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_train_curves, "loss", "train_loss", bs, lr, seq_len, num_epochs)

                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "gru_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [ ]:
# 設定訓練參數
batch_sizes = [4, 8, 16, 32]
learning_rates = [1e-2, 1e-3, 1e-4]
seq_lens = [4, 8, 10, 20, 30, 40]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_gru(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

1D CNN

In [ ]:
# === CNN 模型完整訓練程式 ===
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/CNN"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === CNN 模型 ===
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.model(x)

# === 主訓練與超參數搜尋函數（使用快取） ===
def train_and_search_cnn(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "/content/preprocessed/"
    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = CNNClassifier(num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric_per_fold(fold_lists, metric_name, folder, bs, lr, seq_len, num_epochs):
                    plt.figure(figsize=(10, 6))
                    for fold_idx, fold_metric in enumerate(fold_lists):
                        plt.plot(range(1, num_epochs + 1), fold_metric, label=f"Fold {fold_idx + 1}")  # 無 marker

                    plt.title(f"Combined {metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    if metric_name != 'loss':
                        plt.ylim(0, 1.0)
                    plt.grid(True)
                    plt.legend()
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}_combined_{metric_name}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                plot_metric_per_fold(acc_curves, "accuracy", "accuracy_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_curves, "loss", "loss_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(f1_curves, "f1_score", "f1_score_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(acc_train_curves, "accuracy", "train_accuracy", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_train_curves, "loss", "train_loss", bs, lr, seq_len, num_epochs)

                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "cnn_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [ ]:
# 設定訓練參數
batch_sizes = [4, 8, 16, 32]
learning_rates = [1e-2, 1e-3, 1e-4]
seq_lens = [4, 8, 10, 20, 30, 40]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_cnn(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

TimesNet

In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/TimesNet"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]
# === TimesNet 模型 ===
class TimesBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=3, padding=1)

        # 殘差 skip connection 處理
        if input_dim != hidden_dim:
            self.skip_conv = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=1)
        else:
            self.skip_conv = None

    def forward(self, x):
        residual = x.transpose(1, 2)  # (batch, input_dim, seq_len)

        x = residual
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)

        if self.skip_conv is not None:
            residual = self.skip_conv(residual)

        x = x + residual  # 殘差加法
        x = x.transpose(1, 2)  # 回到 (batch, seq_len, hidden_dim)
        return x

class TimesNetClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.blocks = nn.ModuleList([TimesBlock(input_dim if i==0 else hidden_dim, hidden_dim) for i in range(num_layers)])
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        for block in self.blocks:
            x = block(x)  # 不要再加殘差了，block 內部已處理殘差
        x = x.transpose(1, 2)  # (batch, hidden_dim, seq_len)
        x = self.pool(x).squeeze(-1)  # (batch, hidden_dim)
        out = self.fc(x)
        return out

# === 主訓練與超參數搜尋函數 ===
def train_and_search_timesnet(batch_sizes, learning_rates, seq_lens,
                              num_epochs=100, hidden_dim=64, num_layers=2,
                              num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "/content/preprocessed/"
    RESULTS_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/TimesNet"
    os.makedirs(RESULTS_DIR, exist_ok=True)

    SUB_DIRS = [
        "accuracy_curves", "loss_curves", "f1_score_curves",
        "confusion_matrices", "train_accuracy", "train_loss"
    ]
    for sub in SUB_DIRS:
        os.makedirs(os.path.join(RESULTS_DIR, sub), exist_ok=True)

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = TimesNetClassifier(input_dim=3, hidden_dim=hidden_dim,
                                               num_layers=num_layers, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric_per_fold(fold_lists, metric_name, folder, bs, lr, seq_len, num_epochs):
                    plt.figure(figsize=(10, 6))
                    for fold_idx, fold_metric in enumerate(fold_lists):
                        plt.plot(range(1, num_epochs + 1), fold_metric, label=f"Fold {fold_idx + 1}")  # 無 marker

                    plt.title(f"Combined {metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    if metric_name != 'loss':
                        plt.ylim(0, 1.0)
                    plt.grid(True)
                    plt.legend()
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}_combined_{metric_name}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                # === Plot ===
                plot_metric_per_fold(acc_curves, "accuracy", "accuracy_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_curves, "loss", "loss_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(f1_curves, "f1_score", "f1_score_curves", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(acc_train_curves, "accuracy", "train_accuracy", bs, lr, seq_len, num_epochs)
                plot_metric_per_fold(loss_train_curves, "loss", "train_loss", bs, lr, seq_len, num_epochs)

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    csv_path = os.path.join(RESULTS_DIR, "timesnet_experiment_results.csv")

    # 如果原本有檔案，先讀進來舊的
    if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path)
        df_new = pd.DataFrame(results)
        df_combined = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_combined = pd.DataFrame(results)

    # 儲存合併後的新 dataframe
    df_combined.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df_combined.empty:
        best = df_combined.loc[df_combined['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [ ]:
# 設定訓練參數
batch_sizes = [4, 8, 16, 32]
learning_rates = [1e-2, 1e-3, 1e-4]
seq_lens = [4, 8, 10, 20, 30, 40]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_timesnet(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")